<a href="https://colab.research.google.com/github/komal83/phdthesis/blob/main/AlBERTo_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script predicts the presence of hate speeech or not using AlBERToand taking as inputs Twita tweets as .bz files

In [ ]:
# @title Preparation
!pip install -q keras-bert keras-rectified-adam


In [ ]:
# @title Import from GPU bert ijcai
%tensorflow_version 1.x
import bz2  #use only when importing .bz files
import os
import tensorflow as tf
import keras
from keras_radam import RAdam
from keras_bert import get_custom_objects
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer
import pandas as pd
import tensorflow.keras.backend as K
import sys
from sklearn.metrics import classification_report
from google.colab import drive

In [ ]:
# @title Constants

np.random.seed(999)
SEQ_LEN = 128
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-5

In [ ]:
#@title Drive Mount
#Only if data/models are saved in drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# @title Environment

pretrained_path = '#your path to albert_tweets_uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, <bert_config.json>)
checkpoint_path = os.path.join(pretrained_path, <model.ckpt-1000000>)
vocab_path = os.path.join(pretrained_path, <vocabulary_lower_case_128.txt>)

In [ ]:
# @title Load Basic Model
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,
)

In [ ]:
# @title load train data  


tokenizer = Tokenizer(token_dict)


def load_data_train(path):
    global tokenizer
    indices, sentiments = [], []
    #data= pd.read_csv(path,names=['tweet_id', 'text', 'label'],sep='\t')
    data= pd.read_csv(path,names=['tweet_id', 'text', 'label'])
    for text in tqdm(data.text):
        '''for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in tqdm(os.listdir(folder)):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()'''     
        
        ids, segments = tokenizer.encode(str(text).strip(), max_len=SEQ_LEN)
        indices.append(ids)
    #for sentiment in tqdm(data.label.replace({'no': 0, 'yes': 1})):
    for sentiment in tqdm(data.label):   
        #sentiments.append(sentiment)
        #sentiments.append(eval(sentiment))
        try:
            sentiments.append(int(sentiment))
        except:
            print (sentiment)
            print (type(sentiment))
            sys.exit(0)
    
    items = list(zip(indices, sentiments))
    np.random.shuffle(items)
    indices, sentiments = zip(*items)
    indices = np.array(indices)
    mod = indices.shape[0] % BATCH_SIZE
    if mod > 0:
        indices, sentiments = indices[:-mod], sentiments[:-mod]
    return [indices, np.zeros_like(indices)], np.array(sentiments)



In [ ]:
# @title load test data 
def load_data_test(path):
    global tokenizer
    indices = []
    data= pd.read_csv(path)
    for text in tqdm(data.text): 
        ids, segments = tokenizer.encode(str(text).strip(), max_len=SEQ_LEN)
        indices.append(ids)
    np.random.shuffle(indices)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

In [ ]:
# @title  Load Traing Data
train_x, train_y = load_data_train('<2019_02_k.csv>')
#print(train_x)

100%|██████████| 2000/2000 [00:00<00:00, 883383.32it/s]


In [ ]:
# @title Build Custom Model


inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=2, activation='softmax')(dense)

model = keras.models.Model(inputs, outputs)
model.compile(
    RAdam(lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [ ]:
# @title Initialize Variables


sess = K.get_session()

#sess= K.tensorflow_backend.set_session(sess)


uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [ ]:
# @title Fit

model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)

In [ ]:
#@title Load Test Data
test_x=load_data_test('<40wita_2020-02-02.csv.bz2>')


In [ ]:
# @title Predict
predicts = model.predict(test_x, verbose=True).argmax(axis=-1)
#print(predicts)

In [ ]:
# @title Data Writing predicted labels to input test file
data2= test_x
data2['predicted'] = list(predicts)
#data2.to_csv("<40wita_2020-02-02_prediction.csv>", index=False)

#data2.to_csv('/content/drive/My Drive/COVID-19/40wita/40wita_2020-02-01_predicted.csv.bz2', index=False)